# Imports

In [1]:
import tensorflow as tf
import tensorflow_io as tfio

In [2]:
from IPython.display import Audio

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

In [4]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

In [5]:
from birds.preproc import generate_tensor
from birds.preproc import generate_spectrogram, generate_mel_spectrogram, generate_db_scale_mel_spectrogram
from birds.preproc import one_hot_encode_target

# Dataset

In [6]:
RAW_DATA_PATH = '../raw_data/'
TRAIN_SET_PATH = '../raw_data/data_30s/train/'

In [7]:
metadata_train = pd.read_csv(RAW_DATA_PATH + 'metadata_train.csv')

metadata_train.head(3)

,Recording_ID,Genus,Specific_epithet,Subspecies,English_name,Recordist,Country,Locality,Latitude,Longitude,...,Other_species24,Other_species25,Other_species26,Other_species27,Other_species28,Other_species29,Other_species30,Species,Path,Target
0,447407,Sonus,naturalis,NaN,Soundscape,José Carlos Sires,Spain,"Córdoba, Córdoba, Andalucía",37.9413,-4.8958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonus naturalis,Sonus-naturalis-447407_tens.ogg,0
1,387437,Sonus,naturalis,NaN,Soundscape,José Carlos Sires,Spain,"el planerón, belchite, zaragoza, aragón",41.2784,-0.7328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonus naturalis,Sonus-naturalis-387437_tens.ogg,0
2,383228,Sonus,naturalis,NaN,Soundscape,José Carlos Sires,Spain,"río guadalmellato, córdoba, andalucía",38.0306,-4.6698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonus naturalis,Sonus-naturalis-383228_tens.ogg,0


In [8]:
species_table = metadata_train.copy()[['Species', 'English_name', 'Target']]
species_table = species_table.drop_duplicates()
species_table = species_table.reset_index(drop=True)

species_table.head(3)

,Species,English_name,Target
0,Sonus naturalis,Soundscape,0
1,Fringilla coelebs,Common Chaffinch,1
2,Parus major,Great Tit,2


In [9]:
y_train = pd.read_csv(TRAIN_SET_PATH + 'y_train.csv')

y_train.head(3)

,Path,Target
0,Sonus-naturalis-447407_tens.ogg,0
1,Sonus-naturalis-387437_tens.ogg,0
2,Sonus-naturalis-383228_tens.ogg,0


# Testing functions

In [10]:
path_example = TRAIN_SET_PATH + y_train.loc[50, 'Path']
label_example = y_train.loc[50, 'Target']

print(path_example)
print(label_example)

../raw_data/data_30s/train/Fringilla-coelebs-278566_tens.ogg
1


In [11]:
tensor, label, input_rate, output_rate = generate_tensor(path_example, label_example)

In [12]:
spectrogram, label = generate_spectrogram(path_example, label_example)

In [13]:
mel_spectrogram, label = generate_mel_spectrogram(path_example, label_example)

In [14]:
db_scale_mel_spectrogram, label = generate_db_scale_mel_spectrogram(path_example, label_example)

# Tf spectrograms

In [15]:
split_dict = {
    True: '10s',
    False: '30s'
}
plot_dict = {
    'mel spectrogram': 'log',
    'spectrogram': 'log',
    'db spectrogram': 'common'
}

In [17]:
@interact(
    target=range(1,50), output_rate=[16_000, 44_100, 48_000], split=[True, False], 
    spec_type=['mel spectrogram', 'spectrogram', 'db spectrogram'])
def print_from_target(target, output_rate, split, spec_type):
    
    # Suptitle from target, spectrogram type, plot type
    species_name = species_table[species_table['Target'] == target].loc[:, 'Species'].reset_index(drop=True)[0]
    english_name = species_table[species_table['Target'] == target].loc[:, 'English_name'].reset_index(drop=True)[0]
    suptitle = f"Target {target}: {species_name} ~ {english_name}\noutput rate: {output_rate} - split: {split_dict[split]}\n\n{spec_type.upper()}\n"
    
    # Get 3 first audio files from chosen target
    files_names = list(y_train[y_train['Target'] == target].reset_index(drop=True).loc[0:3, 'Path'])
    files_paths = [TRAIN_SET_PATH + file_name for file_name in files_names]
    
    plt.figure(figsize=(20,5), constrained_layout=True)

    for j in range(3):
        file_name = files_names[j]
        file_path = files_paths[j]
        record_name = file_name.split("_tens.ogg")[0]
        
        # collect tensor information
        tensor, label, input_rate, output_rate = generate_tensor(file_path, target, split=split, output_rate=output_rate)
        
        # choose spectrogram type
        if spec_type == 'spectrogram':
            spectrogram, label = generate_spectrogram(file_path, target, output_rate=output_rate, split=split)
        elif spec_type == 'mel spectrogram':
            spectrogram, label = generate_mel_spectrogram(file_path, target, output_rate=output_rate, split=split)
        elif spec_type == 'db spectrogram':
            spectrogram, label = generate_db_scale_mel_spectrogram(file_path, target, output_rate=output_rate, split=split)
        
        plt.subplot(1, 3, j+1)
        
        # get plot type
        plot_type = plot_dict[spec_type] # 'log' for spec and mel-spec, 'common' for db-spec
        if plot_type == 'common':
            plt.imshow(spectrogram.numpy())
        elif plot_type == 'log':
            plt.imshow(tf.math.log(spectrogram).numpy())

        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        
        title = f"Record n°{j}: {record_name}\nInput rate: {input_rate}\nTensor shape: {tensor.shape}\nSpectrogram shape: {spectrogram.shape} ~ {spectrogram.shape[0] * spectrogram.shape[1]}\n"
        plt.title(title, fontsize=12, loc='left')


    plt.suptitle(suptitle, fontsize=18);


interactive(children=(Dropdown(description='target', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

In [18]:
@interact(target=range(1,50), record=range(3))
def display_audio_from_target(target, record):
    
    files_names = list(y_train[y_train['Target'] == target].reset_index(drop=True).loc[:, 'Path'])
    files_paths = [TRAIN_SET_PATH + file_name for file_name in files_names]
    
    return Audio(files_paths[record])

interactive(children=(Dropdown(description='target', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…